# Merge/split analysis

See [merge_split_epxeriments](merge_split_experiments.ipynb) for inital dev work. From that, I worked out a few things to do with how merges/splits work, summarized here.

1. Clouds smaller than MCSs can be merged/split into an MCS. This is what e.g. the `dstracks.merge_cloudnumber` fields tracks. Likewise, this info is in the `dspixel` data.
2. MCSs can merge split, if the clouds (as above) are large enough to fulfil the MCS criteria.
3. The information about which MCS a daughter MCS split from (or merges into) can be obtained from the `dstracks` data.

Here, I'm going to do some nicer analysis that demonstrates the above for sharing with other people.

* 13/6/22: Initial work

## Analysis 1 (merge/split clouds)

In [1]:
import datetime as dt
import itertools
from pathlib import Path

import dask
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation
import numpy as np
import pandas as pd
from scipy import stats
from scipy import interpolate
import xarray as xr

from mcs_prime import McsTracks, McsTrack, PATHS
from mcs_prime.util import update_progress
%matplotlib inline

/home/users/mmuetz/miniconda3/envs/mcs_prime_env/lib/python3.10/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


KeyboardInterrupt: 

Load all tracks from 2000/6-2020/12

In [ ]:
tracks = tracks = McsTracks.load(PATHS['statsdir'] / 'mcs_tracks_final_extc_20190101.0000_20200101.0000.nc', 
                                 PATHS['pixeldir'])
tracks

In [ ]:
dstrack = tracks.dstracks.isel(tracks=(tracks.dstracks.track_duration > 30) & (tracks.dstracks.track_duration < 40)).isel(tracks=0)
track = McsTrack(dstrack.tracks.values.item(), dstrack, tracks.pixel_data)


# track = tracks.get_track(15477)
# track = tracks.get_track(298)
# track = tracks.get_track(1234)
track

In [ ]:
start = pd.Timestamp(track.dstrack.start_basetime.values).to_pydatetime()
end = pd.Timestamp(track.dstrack.end_basetime.values).to_pydatetime()

In [ ]:
frames = tracks.pixel_data.get_frames(start, end)

In [ ]:
# frames.dspixel.cloudnumber.load()
swath_extent = frames.get_min_max_lon_lat(track.cloudnumber)

In [ ]:
def plot_cloud_merge_split(i):
    cn = track.cloudnumber[i]
    ax.clear()
    ax.contour(frames.dspixel.lon, frames.dspixel.lat, frames.dspixel.cloudnumber[i] == cn, levels=[0.5], colors=['k'])
    ax.set_xlim(swath_extent[:2])
    ax.set_ylim(swath_extent[2:])
    for mcn in track.dstrack.merge_cloudnumber.values[i]:
        if np.isnan(mcn):
            break
        ax.contour(frames.dspixel.lon, frames.dspixel.lat, frames.dspixel.cloudnumber[i] == mcn, levels=[0.5], colors=['g'])
    for scn in track.dstrack.split_cloudnumber.values[i]:
        if np.isnan(scn):
            break
        ax.contour(frames.dspixel.lon, frames.dspixel.lat, frames.dspixel.cloudnumber[i] == scn, levels=[0.5], colors=['r'])

In [ ]:
fig, ax = plt.subplots()

In [ ]:
anim = matplotlib.animation.FuncAnimation(fig, plot_cloud_merge_split, frames=track.duration, interval=500)
anim